In [34]:
import re
import markdown
text = '- Formula: \\( A = P \\left(1 + \\frac{r}{n}\\right)^{nt} \\)'
# text.replace('<', '< ')
# text.replace('\\(', '\\( ')
# text.replace('\\)', ' \\) ')
# text.replace('\\[', '\\[ ')
# text.replace('\\]', ' \\] ')

# text = re.sub(r'\n+([ \t]+\$\$|[ \t]+\\\])', r'\n\1', text)
# text = re.sub(r'(\$\$|\\\[)\n+', r'\1\n', text)
text = re.sub(r'\\([()\[\]+])', r'\\\\\1', text)
html_text = markdown.markdown(text, extensions=['extra', 'smarty'])
print(text)
print(html_text)

- Formula: \\( A = P \left(1 + \frac{r}{n}\right)^{nt} \\)
<ul>
<li>Formula: \( A = P \left(1 + \frac{r}{n}\right)^{nt} \)</li>
</ul>


In [7]:
from notebot2 import md_to_html_for_input

print(md_to_html_for_input('- Formula: \\( A = P \\left(1 + \\frac{r}{n}\\right)^{nt} \\)'))

<ul>
<li>Formula: ( A = P \left(1 + \frac{r}{n}\right)^{nt} )</li>
</ul>


In [8]:
from notebot2 import md_to_html_for_input

print(md_to_html_for_input('\\( A = P \\left(1 + \\frac{r}{n}\\right)^{nt} \\)'))

<p>( A = P \left(1 + \frac{r}{n}\right)^{nt} )</p>


In [9]:
md_to_html_for_input('<li>Ideal Gas Law: \( PV = nRT \)</li>')

<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
C:\Users\harold\AppData\Local\Temp\ipykernel_20256\922698136.py:1: SyntaxWarning: invalid escape sequence '\('
  md_to_html_for_input('<li>Ideal Gas Law: \( PV = nRT \)</li>')


'<li>Ideal Gas Law: \\( PV = nRT \\)</li>'

In [20]:
md_to_html_for_input('\(e^{i}\)')

<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
C:\Users\harold\AppData\Local\Temp\ipykernel_20256\332597510.py:1: SyntaxWarning: invalid escape sequence '\('
  md_to_html_for_input('\(e^{i}\)')


'<p>(e^{i})</p>'

In [23]:
print('\\(e^{i\\pi} + 1 = 0\\)')

\(e^{i\pi} + 1 = 0\)


In [29]:
import os
import re
import openai
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')
if api_key is None:
    key = input("Please input your api key: ").strip()
    api_key = clean_string(key)

client = openai.AzureOpenAI(
    azure_endpoint=os.getenv('OPENAI_API_ENDPOINT', "https://api.hku.hk"),
    api_key=api_key,
    api_version="2025-01-01-preview"
)

def history_default():
    return [{"role": "system", "content": "You are a helpful assistant who helps to write notes. You may also answer user's questions if they have query. You write or correct notes according to the user's input. Keep your response concise. Answer directly without any polite phrases or conclusions. You may have a title, but don't write any openings before the title, but after the title. If there is title, use <h1> (the largest heading) for the title. Use latex for any math symbols. Use 4 spaces for any indentations. Better use <ol> and <ul> tags for lists. Use HTML tags for tables."}]

def select_model(model_input):
    match model_input:
        case 'GPT-4.1-nano': return 'gpt-4.1-nano'
        case 'GPT-4.1-mini': return 'gpt-4.1-mini'
        case 'GPT-4.1': return 'gpt-4.1'
        case 'o4-mini': return 'o4-mini'
        case 'GPT-4o': return 'gpt-4o'
        case 'GPT-4': return 'gpt-4'
        case 'GPT-3.5': return 'gpt-35-turbo'
        case _: return model_input

def chatbot(message, model, prompt, temperature=0.7, history=history_default(), defining=False):
    model=select_model(model)
    temperature = 1 if model == 'o4-mini' else temperature
    
    history[0]['content'] = prompt
    new_prompt = {"role": "user", "content": message}
    history.append(new_prompt)
    # completion = openai.ChatCompletion.create(engine="chatgpt", messages=history)
    response = client.chat.completions.create(model=model, messages=history, temperature=temperature)
    answer = response.choices[0].message.content
    if defining: history.pop()
    elif not defining:
        history.append({"role": "assistant", "content": answer})
    return response, history, answer

In [30]:
response, history, answer = chatbot("Hello", "o4-mini", "You are a helpful assistant who helps to write notes.")